This notebook is a part of motokimura's baseline solution for the [Solafune Identifying Deforestation Drivers competition](https://solafune.com/competitions/68ad4759-4686-4bb3-94b8-7063f755b43d?menu=about&tab=overview).
See https://github.com/motokimura/solafune_deforestation_baseline for the complete code.

> cf. @solafune (https://solafune.com) Use for any purpose other than participation in the competition or commercial use is prohibited. If you would like to use them for any of the above purposes, please contact us.

### Description

**By running this notebook, you will achieve a score of around 0.533 on the public leaderboard.**

This notebook trains a U-Net model for 4-class segmentation (`grassland_shrubland`, `logging`, `mining`, and `plantation`) and generates a submission JSON file for the evaluation images from the output from the U-Net model.

The submission JSON file is saved to `data/submission.json`.

Before running this notebook, you have to run `generate_masks.ipynb` to generate `.npy` files used for training
(`generate_masks.ipynb` is available from https://github.com/motokimura/solafune_deforestation_baseline).

### Requirements

#### Datasets

Organize the dataset as follows:

```
data/
├── evaluation_images/
│   ├── evaluation_0.tif
│   ├── evaluation_1.tif
│   ├── evaluation_2.tif
│   ├── ...
├── train_images/
│   ├── train_0.tif
│   ├── train_1.tif
│   ├── train_2.tif
│   ├── ...
├── train_masks/
│   ├── train_0.npy
│   ├── train_1.npy
│   ├── train_2.npy
│   ├── ...
```

`evaluation_images` and `train_images` can be downloaded from the competition page.

`train_masks` can be generated by running `generate_masks.ipynb` available from https://github.com/motokimura/solafune_deforestation_baseline.

#### Libraries

Please install the python packages imported the cell below.


In [74]:
import json
import os

from pathlib import Path

import albumentations as albu  # tested with 1.4.24
import imagecodecs
import numpy as np  # tested with 1.26.4
import pandas as pd
import pytorch_lightning as pl  # tested with 2.5.0.post0
import segmentation_models_pytorch as smp  # tested with 0.3.4
import sklearn
import tensorboard
import tifffile
import timm  # tested with 0.9.7
import torch  # tested with 2.5.1

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from rasterio import features
from shapely.geometry import Polygon, shape
from skimage import measure
from timm.optim import create_optimizer_v2
from timm.scheduler import create_scheduler_v2
from tqdm import tqdm

Torch version: 2.6.0+cu118
CUDA available: True
CUDA devices: 1
GPU Name: NVIDIA GeForce GTX 1060 6GB


In [75]:
seed_everything(42)

Seed set to 42


42

In [76]:
import sys
from pathlib import Path

# Move up from the notebook directory to project root
BASE_DIR = Path.cwd().resolve().parent.parent  # Moves up two levels to `solafune-deforestation`
sys.path.insert(0, str(BASE_DIR))  # Add project root to Python path

# Import paths dynamically
from src.paths import DATASET_PATH, TRAIN_IMAGES_PATH, EVAL_IMAGES_PATH, TRAIN_ANNOTATIONS_PATH

print("DATASET_PATH:", DATASET_PATH)  # Debugging


class_names = ["grassland_shrubland", "logging", "mining", "plantation"]

epochs = 200

DATASET_PATH: C:\Users\Brage\Desktop\skole\solafune-deforestation\data


### Define dataset class to load images and masks for training and validation

In [77]:
def load_mask(mask_path):
    mask = np.load(mask_path)  # (4, H, W), uint8
    assert mask.shape == (4, 1024, 1024)
    mask = mask.transpose(1, 2, 0)  # (H, W, 4)
    return mask.astype(np.float32) / 255.0  # normalize to [0, 1]


def load_image(image_path):
    image = tifffile.imread(image_path)  # (H, W, 12), float64
    assert image.shape == (1024, 1024, 12)
    image = np.nan_to_num(image)  # replace NaN with 0
    return image.astype(np.float32)


def normalize_image(image):
    # mean of train images
    mean = np.array(
        [
            285.8190561180765,
            327.22091430696577,
            552.9305957826701,
            392.1575148484924,
            914.3138803812591,
            2346.1184507500043,
            2884.4831706095824,
            2886.442429854111,
            3176.7501338557763,
            3156.934442092072,
            1727.1940075511282,
            848.573373995044,
        ],
        dtype=np.float32
    )

    # std of train images
    std = np.array(
        [
            216.44975668759372,
            269.8880248304874,
            309.92790753407064,
            397.45655590699,
            400.22078920482215,
            630.3269651264278,
            789.8006920468097,
            810.4773696969773,
            852.9031432100967,
            807.5976198303886,
            631.7808113929271,
            502.66788721341396,
        ],
        dtype=np.float32
    )
    
    mean = mean.reshape(12, 1, 1)
    std = std.reshape(12, 1, 1)

    return (image - mean) / std


class TrainValDataset(torch.utils.data.Dataset):
    def __init__(self, data_root, sample_indices, augmentations=None):
        self.image_paths, self.mask_paths = [], []
        for i in sample_indices:
            self.image_paths.append(data_root / "train_images" / f"train_{i}.tif")
            self.mask_paths.append(data_root / "train_masks" / f"train_{i}.npy")
        self.augmentations = augmentations

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        sample = {
            "image": load_image(self.image_paths[idx]),
            "mask": load_mask(self.mask_paths[idx]),
        }

        if self.augmentations is not None:
            sample = self.augmentations(**sample)

        sample["image"] = sample["image"].transpose(2, 0, 1)  # (12, H, W)
        sample["mask"] = sample["mask"].transpose(2, 0, 1)  # (4, H, W)

        sample["image"] = normalize_image(sample["image"])

        # add metadata
        sample["image_path"] = str(self.image_paths[idx])
        sample["mask_path"] = str(self.mask_paths[idx])

        return sample

### Define U-Net model using pytorch-lightning

In [78]:
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # prepare segmentation model
        self.model = smp.create_model(
            arch="unet",
            encoder_name="tu-tf_efficientnetv2_s",  # use `tf_efficientnetv2_s` from timm
            encoder_weights="imagenet",  # always starts from imagenet pre-trained weight
            in_channels=12,
            classes=4,
        )

        # prepare loss functions
        self.dice_loss_fn = smp.losses.DiceLoss(mode=smp.losses.MULTILABEL_MODE, from_logits=True)
        self.bce_loss_fn = smp.losses.SoftBCEWithLogitsLoss(smooth_factor=0.0)

        self.training_step_outputs = []
        self.validation_step_outputs = []
    
    def forward(self, image):
        # assuming image is already normalized
        return self.model(image)  # logits

    def shared_step(self, batch, stage):
        image = batch["image"]
        mask = batch["mask"]

        logits_mask = self.forward(image)

        loss = self.dice_loss_fn(logits_mask, mask) + self.bce_loss_fn(logits_mask, mask)

        # count tp, fp, fn, tn for each class to compute validation metrics at the end of epoch
        thresh = 0.5
        prob_mask = logits_mask.sigmoid()
        tp, fp, fn, tn = smp.metrics.get_stats(
            (prob_mask > thresh).long(),
            mask.long(),
            mode=smp.losses.MULTILABEL_MODE,
        )  # each of tp, fp, fn, tn is a tensor of shape (batch_size, num_classes) and of type long

        output = {
            "loss": loss.detach().cpu(),
            "tp": tp.detach().cpu(),
            "fp": fp.detach().cpu(),
            "fn": fn.detach().cpu(),
            "tn": tn.detach().cpu(),
        }
        if stage == "train":
            self.training_step_outputs.append(output)
        else:
            self.validation_step_outputs.append(output)

        return loss

    def training_step(self, batch, batch_idx):
        return self.shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self.shared_step(batch, "val")

    def shared_epoch_end(self, outputs, stage):
        def log(name, tensor, prog_bar=False):
            self.log(f"{stage}/{name}", tensor.to(self.device), sync_dist=True, prog_bar=prog_bar)

        # aggregate loss
        loss = torch.stack([x["loss"] for x in outputs]).mean()
        log("loss", loss, prog_bar=True)

        # aggregate tp, fp, fn, tn to compose F1 score for each class
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])

        f1_scores = {}
        for i, class_name in enumerate(class_names):
            f1_scores[class_name] = smp.metrics.f1_score(tp[:, i], fp[:, i], fn[:, i], tn[:, i], reduction="macro-imagewise")
            log(f"f1/{class_name}", f1_scores[class_name], prog_bar=False)

        f1_avg = torch.stack([v for v in f1_scores.values()]).mean()
        log("f1", f1_avg, prog_bar=True)
    
    def on_train_epoch_end(self):
        self.shared_epoch_end(self.training_step_outputs, "train")
        self.training_step_outputs.clear()

    def on_validation_epoch_end(self):
        self.shared_epoch_end(self.validation_step_outputs, "val")
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        # optimizer
        optimizer = create_optimizer_v2(
            self.parameters(),
            opt="adamw",
            lr=1e-4,
            weight_decay=1e-2,
            filter_bias_and_bn=True,  # filter out bias and batchnorm from weight decay
        )

        # lr scheduler
        scheduler, _ = create_scheduler_v2(
            optimizer,
            sched="cosine",
            num_epochs=epochs,
            min_lr=0.0,
            warmup_lr=1e-5,
            warmup_epochs=0,
            warmup_prefix=False,
            step_on_epochs=True,
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch",
            },
        }

    def lr_scheduler_step(self, scheduler, metric):
        # workaround for timm's scheduler:
        # https://github.com/Lightning-AI/lightning/issues/5555#issuecomment-1065894281
        scheduler.step(epoch=self.current_epoch)  # timm's scheduler need the epoch value

### Prepare trainer of pytorch-lightning

In [79]:
train_output_dir = DATASET_PATH / "training_result"

# split train_images into train-set and val-set
sample_indices = list(range(176))  # train_0.tif to train_175.tif
train_indices, val_indices = sklearn.model_selection.train_test_split(sample_indices, test_size=0.2, random_state=42)

# augmentations applied only to train-set
augmentations = albu.Compose(
[        
        # shift, scale, and rotate
        albu.ShiftScaleRotate(
            p=0.5,
            shift_limit=0.0625,
            scale_limit=0.1,
            rotate_limit=15,
            border_mode=0,  # constant border
            value=0,
            mask_value=0,
            interpolation=2,  # bicubic
        ),
        # random crop
        albu.RandomCrop(
            p=1,
            width=512,
            height=512,
        ),
        # flip, transpose, and rotate90
        albu.Resize(512, 512),  # Resize images
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.Transpose(p=0.5),
        albu.RandomRotate90(p=0.5),
    ]
)

# prepare data loaders
num_workers = min(2, os.cpu_count())  # Adaptive worker selection
train_loader = torch.utils.data.DataLoader(
    TrainValDataset(
        DATASET_PATH,
        train_indices,
        augmentations=augmentations,
    ),
    batch_size=4,  # Reduce batch size
    num_workers=0,  # Reduce workers to avoid memory overhead
    shuffle=True,
)

val_loader = torch.utils.data.DataLoader(
    TrainValDataset(
        DATASET_PATH,
        val_indices,
        augmentations=None,
    ),
    batch_size=2,  # Reduce validation batch size
    num_workers=0,
    shuffle=False,
)

# prepare trainer
trainer = Trainer(
    max_epochs=epochs,
    callbacks = [
        # save model with best validation F1 score
        ModelCheckpoint(
            dirpath=train_output_dir,
            filename="best_f1_05",
            save_weights_only=True,
            save_top_k=1,
            monitor="val/f1",
            mode="max",
            save_last=False,
        ),
        LearningRateMonitor(logging_interval="step"),
    ],
    logger=[TensorBoardLogger(train_output_dir, name=None)],
    precision="16-mixed",
    deterministic=True,
    benchmark=False,
    sync_batchnorm=False,
    check_val_every_n_epoch=5,
    default_root_dir=os.getcwd(),
    accelerator="gpu",
    devices=[0,],
    strategy="auto",
    log_every_n_steps=5,
    accumulate_grad_batches=4,
)

# prepare model
model = Model()

Using 16bit Automatic Mixed Precision (AMP)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


### Start training!

With the default setting, 10 GB of GPU memory is required. To reduce the memory usage, you can decrease the batch size.

The trained model is saved as `data/training_result/best_f1_05.ckpt`.

Tensorboard logs are also saved under `data/training_result/version_xx`.

**The execution often does not finish even after reaching 200 epochs. In that case, you can stop the execution manually and just proceed to the next cell (do not restart the notebook!).**

In [80]:
# start training
trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name         | Type                  | Params | Mode 
---------------------------------------------------------------
0 | model        | Unet                  | 22.1 M | train
1 | dice_loss_fn | DiceLoss              | 0      | train
2 | bce_loss_fn  | SoftBCEWithLogitsLoss | 0      | train
---------------------------------------------------------------
22.1 M    Trainable params
0         Non-trainable params
22.1 M    Total params
88.383    Total estimated model params size (MB)
759       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


### Compute the evaluation metric for the validation set

Before predicting the evaluation images, let's predict the validation set and compute the evaluation metric.

This may be useful to check if the model is trained properly and to tune the parameters for post-processing (e.g., score threshold, etc.).

In [81]:
def run_inference(model, loader, pred_output_dir):
    pred_output_dir = Path(pred_output_dir)
    pred_output_dir.mkdir(exist_ok=True, parents=True)

    for batch in tqdm(loader):
        img = batch["image"].cuda()

        with torch.no_grad():
            logits_mask = model(img)
            prob_mask = logits_mask.sigmoid()

        # save prob mask as numpy array
        for i in range(img.size(0)):
            file_name = os.path.basename(batch["image_path"][i])
            prob_mask_i = prob_mask[i].cpu().numpy()  # (4, 1024, 1024)

            np.save(
                pred_output_dir / file_name.replace(".tif", ".npy"),
                prob_mask_i.astype(np.float16),
            )

In [84]:
# load best checkpoint and run inference on val-set
del model

model = Model()
model.load_state_dict(torch.load(train_output_dir / "best_f1_05-v2.ckpt")["state_dict"])
model = model.cuda()
model.eval()

val_pred_dir = DATASET_PATH / "val_preds"
run_inference(model, val_loader, val_pred_dir)

100%|██████████| 18/18 [00:23<00:00,  1.29s/it]


In [85]:
def compute_f1_score(pred_mask, truth_mask):
    # `pred_mask` is a binary numpy array of shape (H, W) = (1024, 1024)
    # `truth_mask` is a binaru numpy array of shape (H, W) = (1024, 1024)
    assert pred_mask.shape == (1024, 1024), f"{pred_mask.shape=}"
    assert truth_mask.shape == (1024, 1024), f"{truth_mask.shape=}"

    tp = ((pred_mask > 0) & (truth_mask > 0)).sum()
    fp = ((pred_mask > 0) & (truth_mask == 0)).sum()
    fn = ((pred_mask == 0) & (truth_mask > 0)).sum()
    precision = tp / (tp + fp) if tp + fp > 0 else 1  # if no prediction, precision is considered as 1
    recall = tp / (tp + fn) if tp + fn > 0 else 1  # if no ground truth, recall is considered as 1
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0  # if either precision or recall is 0, f1 is 0

    return f1


score_thresh = 0.7  # threshold to binarize the prediction mask
min_area = 10000  # if the predicted area of a class is less than this, submit an zero mask because small predicted areas are often false positives

val_f1_scores = {}
for idx in sorted(val_indices):
    fn = f"train_{idx}"
    # prepare prediction mask
    pred_mask = np.load(val_pred_dir / f"{fn}.npy")  # (4, 1024, 1024)
    pred_mask = pred_mask > score_thresh  # binarize
    # prepare ground truth mask
    truth_mask = np.load(DATASET_PATH / "train_masks" / f"{fn}.npy")  # (4, 1024, 1024)
    # compute f1 score for each class
    val_f1_scores[fn] = {}
    for i, class_name in enumerate(class_names):
        pred_for_a_class = pred_mask[i]
        if pred_for_a_class.sum() < min_area:
            pred_for_a_class = np.zeros_like(pred_for_a_class)  # set all to zero if the predicted area is less than `min_area`
        val_f1_scores[fn][class_name] = compute_f1_score(pred_for_a_class, truth_mask[i])
val_f1_scores = pd.DataFrame(val_f1_scores).T

# add a column for average of all the 4 classes
val_f1_scores["all_classes"] = val_f1_scores.mean(axis=1)
# add a row for average of all the val images
val_f1_scores.loc["all_images"] = val_f1_scores.mean()

print(f"val f1 score: {val_f1_scores.loc['all_images', 'all_classes']:.4f}")

val_f1_scores

val f1 score: 0.6974


,grassland_shrubland,logging,mining,plantation,all_classes
train_9,0.943992,1.000000,1.000000,1.000000,0.985998
train_12,1.000000,1.000000,1.000000,0.000000,0.750000
train_15,0.000000,1.000000,1.000000,0.606809,0.651702
train_16,0.438303,1.000000,1.000000,0.933849,0.843038
train_18,0.593364,1.000000,1.000000,0.956783,0.887537
train_19,0.000000,1.000000,1.000000,0.549777,0.637444
train_24,1.000000,1.000000,1.000000,0.910089,0.977522
train_29,0.000000,1.000000,1.000000,0.523292,0.630823
train_30,0.988026,1.000000,1.000000,1.000000,0.997006
train_31,0.608888,0.000000,0.000000,0.000000,0.152222


### Predict the evaluation images and generate a submission JSON file

Let's predict the evaluation images as already done with the validation set, and generate a submission JSON file.

The submission JSON file will be saved as `data/submission.json`.

In [86]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data_root):
        self.image_paths = []
        for i in range(118):  # evaluation_0.tif to evaluation_117.tif
            self.image_paths.append(data_root / "evaluation_images" / f"evaluation_{i}.tif")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        sample = {
            "image": load_image(self.image_paths[idx]),
        }

        sample["image"] = sample["image"].transpose(2, 0, 1)  # (12, H, W)
        sample["image"] = normalize_image(sample["image"])

        # add metadata
        sample["image_path"] = str(self.image_paths[idx])

        return sample

In [88]:
test_loader = torch.utils.data.DataLoader(
    TestDataset(DATASET_PATH),
    batch_size=4,
    num_workers=0,
    shuffle=False,
)

test_pred_dir = DATASET_PATH / "test_preds"
run_inference(model, test_loader, test_pred_dir)

100%|██████████| 30/30 [01:17<00:00,  2.57s/it]


`detect_polygons()` below extracts isolated areas as polygons from the predicted mask.

The point is `min_area` parameter to filter out small areas. Small predicted areas are often false positives which decrease the evaluation score.

In [89]:
def detect_polygons(pred_dir, score_thresh, min_area):
    pred_dir = Path(pred_dir)
    pred_paths = list(pred_dir.glob("*.npy"))
    pred_paths = sorted(pred_paths)

    polygons_all_imgs = {}
    for pred_path in tqdm(pred_paths):
        polygons_all_classes = {}

        mask = np.load(pred_path)  # (4, 1024, 1024)
        mask = mask > score_thresh  # binarize
        for i, class_name in enumerate(class_names):
            mask_for_a_class = mask[i]
            if mask_for_a_class.sum() < min_area:
                mask_for_a_class = np.zeros_like(mask_for_a_class)  # set all to zero if the predicted area is less than `min_area`

            # extract polygons from the binarized mask
            label = measure.label(mask_for_a_class, connectivity=2, background=0).astype(np.uint8)
            polygons = []
            for p, value in features.shapes(label, label):
                p = shape(p).buffer(0.5)
                p = p.simplify(tolerance=5.0)
                polygons.append(p)
            polygons_all_classes[class_name] = polygons
        polygons_all_imgs[pred_path.name.replace(".npy", ".tif")] = polygons_all_classes

    return polygons_all_imgs

In [90]:
test_pred_polygons = detect_polygons(test_pred_dir, score_thresh=score_thresh, min_area=min_area)

submission_save_path = DATASET_PATH / f"submission.json"

images = []
for img_id in range(118):  # evaluation_0.tif to evaluation_117.tif
    annotations = []
    for class_name in class_names:
        for poly in test_pred_polygons[f"evaluation_{img_id}.tif"][class_name]:
            seg: list[float] = []  # [x0, y0, x1, y1, ..., xN, yN]
            for xy in poly.exterior.coords:
                seg.extend(xy)

            annotations.append({"class": class_name, "segmentation": seg})

    images.append({"file_name": f"evaluation_{img_id}.tif", "annotations": annotations})

with open(submission_save_path, "w", encoding="utf-8") as f:
    json.dump({"images": images}, f, indent=4)

100%|██████████| 118/118 [00:47<00:00,  2.49it/s]
